# Make perturbed data sets

Gareth Funning, University of California, Riverside

In [33]:
import numpy as np
import matplotlib.pyplot as plt

In [34]:
data = np.loadtxt('./elazig_asc.okinv', delimiter=' ') # or wherever the file is on your system!
outroot = 'elazig_asc_'

In [37]:
# covariance function = maxvar*exp(-b*r)
maxvar =  4.92278e-5

# expcos parameters: cov =maxvar*exp(-alpha*r)*cos(beta*r)
alpha = 0.000137739 
beta = -7.46883e-09

N = 5 # number of perturbed data sets required

# unpack what you've got
x = data[:,0]*1000  # convert x coord from km to m
y = data[:,1]*1000  # convert y coord from km to m
pha = data[:,2]

# calculate distance between points in a matrix
xx1, xx2 = np.meshgrid(x,x) # matrices containing coordinates of points
yy1, yy2 = np.meshgrid(y,y)
rgrid = np.sqrt((xx1-xx2)**2+(yy1-yy2)**2)

# and evaluate the vcm on that grid
vcm = maxvar*np.exp(-alpha*rgrid)*np.cos(beta*rgrid)

# Calculate correlated noise using Cholesky Decomposition

# 1. create matrix of N gaussian noise vectors length n (on it's side!) 
n = len(pha);
Z = np.random.normal(0,1,[N,n])

# 2. chol decomp on vcm
V = np.linalg.cholesky(vcm) # upper triangular part of cholesky [V'V=vcm]

# 3. Create matrix X containing N correlated noisevecotrs length n (on it's side!)
X = np.matmul(Z,V);
X = X.T; # transpose to make it N vectors of length n the right way up

# 4. add back to original data
temp, manypha = np.meshgrid(np.ones(N),pha); # make multiple copies of original data in matrix
pha_pets = manypha+X;

# write out perturbed files

for i in np.arange(0,N):
    # assemble a filename
    outname = '{0:s}{1:03d}.okinv'.format(outroot,i)
    print('writing '+outname)
    # assemble the data
    okinv_data=np.column_stack((data[:,0],data[:,1],pha_pets[:,i],data[:,3],data[:,4],data[:,5],np.arange(0,n)))
    # write the file
    np.savetxt(outname,okinv_data,fmt='%f %f %f %f %f %f %d')

writing elazig_asc_000.okinv
writing elazig_asc_001.okinv
writing elazig_asc_002.okinv
writing elazig_asc_003.okinv
writing elazig_asc_004.okinv
